In [1]:
# Scenario Generation with Copulas 
# 
# Hugo S. de Araujo
# Nov. 14th, 2022 | Mays Group | Cornell University
################################################################################

#=======================================================================
PROJECT SETUP
=======================================================================#
using Pkg
Pkg.activate("copulas");
Pkg.instantiate();
# Import "here" function. Wrapper to allow easy path concatenation.
include(joinpath(@__DIR__, "functions", "fct_here.jl"))

# Import all required packages. 
begin
    # using AWSS3
    using CSV
    using DataFrames
    using Dates
    using DelimitedFiles
    using Distributions
    using HDF5
    using JuliaFormatter
    using LaTeXStrings
    using LinearAlgebra
    using LinearSolve
    #using Measures
    using Random
    using RCall
    using Revise
    using Statistics
    using StatsBase
    #using StatsPlots
    using OhMyREPL
    using Plots
    #using PrettyTables
    using Tables
    using TSFrames
    using TimeZones
end

# Include functions 
#= functions_dirpath = joinpath(pwd(),"src", "functions");
function_paths = readdir(functions_dirpath, join=true);
function_index = occursin.(".jl", function_paths);
functions_only = function_paths[function_index];

for str in functions_only
    include(str)
end =#

include(here("src", "functions", "fct_bind_historical_forecast.jl"));
include(here("src", "functions", "fct_compute_hourly_average_actuals.jl"));
include(here("src", "functions", "fct_compute_landing_probability.jl"));
include(here("src", "functions", "fct_convert_hours_2018.jl"));
include(here("src", "functions", "fct_convert_ISO_standard.jl"));
include(here("src", "functions", "fct_convert_land_prob_to_data.jl"));
include(here("src", "functions", "fct_generate_probability_scenarios.jl"));
include(here("src", "functions", "fct_getplots.jl"));
include(here("src", "functions", "fct_plot_correlation_heatmap.jl"));
include(here("src", "functions", "fct_plot_historical_landing.jl"));
include(here("src", "functions", "fct_plot_historical_synthetic_autocorrelation.jl"));
include(here("src", "functions", "fct_plot_correlogram_landing_probability.jl"));
include(here("src", "functions", "fct_plot_scenarios_and_actual.jl"));
include(here("src", "functions", "fct_read_h5_file.jl"));
include(here("src", "functions", "fct_read_input_file.jl"));
include(here("src", "functions", "fct_transform_landing_probability.jl"));
include(here("src", "functions", "fct_write_percentiles.jl"));
#=======================================================================
READ INPUT FILE
=======================================================================#
input_file_path = here("src\\copulas.txt")

# XXX Needs to be updated to be a hardcoded instead of reading in a text file
data_type,
scenario_length,
number_of_scenarios,
scenario_hour,
scenario_day,
scenario_month,
scenario_year,
read_locally,
historical_load,
forecast_load,
historical_solar,
forecast_da_solar,
forecast_2da_solar,
historical_wind,
forecastd_da_wind,
forecast_2da_wind,
write_percentile = read_input_file(input_file_path);

#=======================================================================
READ INPUT DATA: ARPA-E PERFORM PROJECT H5 FILES
=======================================================================#
# Function that reads the .h5 file and binds the time index and the actuals/fore-
# cast values into a single dataframe.

# Load data
load_actuals_raw = read_h5_file(here("data", historical_load), "load");
load_forecast_raw = read_h5_file(here("data", "ercot_BA_load_forecast_day_ahead_2018.h5"), "load", false);

# Solar data
solar_actuals_raw = read_h5_file(here("data", "ercot_BA_solar_actuals_Existing_2018.h5"), "solar");
solar_forecast_dayahead_raw = read_h5_file(here("data", "ercot_BA_solar_forecast_day_ahead_existing_2018.h5"), "solar", false);
solar_forecast_2dayahead_raw = read_h5_file(here("data", "ercot_BA_solar_forecast_2_day_ahead_existing_2018.h5"), "solar", false);

# Wind data
wind_actuals_raw = read_h5_file(here("data", "ercot_BA_wind_actuals_Existing_2018.h5"), "wind");
wind_forecast_dayahead_raw = read_h5_file(here("data", "ercot_BA_wind_forecast_day_ahead_existing_2018.h5"), "wind", false);
wind_forecast_2dayahead_raw = read_h5_file(here("data", "ercot_BA_wind_forecast_2_day_ahead_existing_2018.h5"), "wind", false);

#=======================================================================
Compute the hourly average for the actuals data
=======================================================================#
# Load
aux = compute_hourly_average_actuals(load_actuals_raw);
load_actual_avg_raw = DataFrame();
time_index = aux[:, :Index];
avg_actual = aux[:, :values_mean];
load_actual_avg_raw[!, :time_index] = time_index;
load_actual_avg_raw[!, :avg_actual] = avg_actual;

# Solar
aux = compute_hourly_average_actuals(solar_actuals_raw);
time_index = aux[:, :Index];
avg_actual = aux[:, :values_mean];
solar_actual_avg_raw = DataFrame();
solar_actual_avg_raw[!, :time_index] = time_index;
solar_actual_avg_raw[!, :avg_actual] = avg_actual;

# Wind
aux = compute_hourly_average_actuals(wind_actuals_raw);
time_index = aux[:, :Index];
avg_actual = aux[:, :values_mean];
wind_actual_avg_raw = DataFrame();
wind_actual_avg_raw[!, :time_index] = time_index;
wind_actual_avg_raw[!, :avg_actual] = avg_actual;

  Activating project at `c:\Users\ks885\Documents\aa_research\Modeling\norta_scenarios\copulas\src\copulas`

Do the same checks on unique forecast and issue to see if daylight savings time is treated differently or the same (i.e. also missing)

lets start by just printing out the unique issue and forecast times and checking that the daylight savings is there and 

In [2]:
load_forecast_raw

Row,forecast_time,issue_time,p_1,p_2,p_3,p_4,p_5,p_6,p_7,p_8,p_9,p_10,p_11,p_12,p_13,p_14,p_15,p_16,p_17,p_18,p_19,p_20,p_21,p_22,p_23,p_24,p_25,p_26,p_27,p_28,p_29,p_30,p_31,p_32,p_33,p_34,p_35,p_36,p_37,p_38,p_39,p_40,p_41,p_42,p_43,p_44,p_45,p_46,p_47,p_48,p_49,p_50,p_51,p_52,p_53,p_54,p_55,p_56,p_57,p_58,p_59,p_60,p_61,p_62,p_63,p_64,p_65,p_66,p_67,p_68,p_69,p_70,p_71,p_72,p_73,p_74,p_75,p_76,p_77,p_78,p_79,p_80,p_81,p_82,p_83,p_84,p_85,p_86,p_87,p_88,p_89,p_90,p_91,p_92,p_93,p_94,p_95,p_96,p_97,p_98,⋯
,DateTime,DateTime,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,⋯
1,2018-01-01T00:00:00,2017-12-29T18:00:00,35801.5,36883.4,37569.9,38086.3,38506.3,38863.8,39177.3,39458.0,39713.3,39948.2,40166.6,40371.2,40564.1,40746.9,40921.1,41087.7,41247.6,41401.6,41550.3,41694.3,41834.0,41969.9,42102.2,42231.4,42357.7,42481.3,42602.5,42721.4,42838.3,42953.4,43066.7,43178.4,43288.7,43397.6,43505.4,43612.0,43717.6,43822.3,43926.1,44029.2,44131.6,44233.4,44334.7,44435.5,44536.0,44636.1,44736.0,44835.6,44935.2,45034.7,45134.2,45233.8,45333.4,45433.3,45533.4,45633.9,45734.7,45836.0,45937.8,46040.2,46143.3,46247.1,46351.8,46457.4,46564.0,46671.8,46780.7,46891.0,47002.7,47116.0,47231.1,47348.0,47466.9,47588.1,47711.7,47838.0,47967.2,48099.5,48235.4,48375.1,48519.0,48667.8,48821.8,48981.7,49148.3,49322.5,49505.3,49698.2,49902.8,50121.1,50356.1,50611.4,50892.1,51205.6,51563.1,51983.1,52499.5,53186.0,⋯
2,2018-01-01T00:00:00,2017-12-30T18:00:00,48101.3,48726.5,49123.2,49421.6,49664.4,49871.0,50052.1,50214.3,50361.8,50497.6,50623.8,50742.0,50853.5,50959.2,51059.8,51156.1,51248.5,51337.5,51423.4,51506.6,51587.4,51665.9,51742.3,51817.0,51890.0,51961.4,52031.4,52100.2,52167.7,52234.2,52299.7,52364.2,52428.0,52490.9,52553.2,52614.8,52675.8,52736.3,52796.3,52855.9,52915.1,52973.9,53032.4,53090.7,53148.7,53206.6,53264.3,53321.9,53379.4,53436.9,53494.4,53552.0,53609.6,53667.3,53725.1,53783.2,53841.5,53900.0,53958.8,54018.0,54077.6,54137.6,54198.1,54259.1,54320.7,54382.9,54445.9,54509.6,54574.2,54639.7,54706.2,54773.7,54842.5,54912.5,54983.9,55056.9,55131.5,55208.0,55286.5,55367.2,55450.4,55536.4,55625.4,55717.8,55814.1,55914.7,56020.4,56131.8,56250.1,56376.3,56512.0,56659.6,56821.8,57002.9,57209.5,57452.2,57750.7,58147.3,⋯
3,2018-01-01T01:00:00,2017-12-29T18:00:00,38623.5,39760.1,40481.1,41023.6,41464.8,41840.4,42169.7,42464.5,42732.7,42979.5,43208.9,43423.8,43626.4,43818.5,44001.4,44176.4,44344.4,44506.2,44662.4,44813.7,44960.4,45103.1,45242.1,45377.8,45510.5,45640.3,45767.6,45892.6,46015.4,46136.2,46255.2,46372.6,46488.4,46602.9,46716.1,46828.0,46939.0,47048.9,47158.0,47266.3,47373.9,47480.8,47587.2,47693.1,47798.6,47903.8,48008.7,48113.4,48218.0,48322.5,48427.0,48531.6,48636.3,48741.3,48846.4,48951.9,49057.9,49164.3,49271.2,49378.8,49487.1,49596.1,49706.1,49817.0,49929.0,50042.2,50156.6,50272.5,50389.8,50508.9,50629.7,50752.5,50877.5,51004.8,51134.6,51267.2,51402.9,51541.9,51684.7,51831.4,51982.6,52138.9,52300.6,52468.6,52643.6,52826.6,53018.7,53221.3,53436.2,53665.6,53912.4,54180.5,54475.4,54804.7,55180.2,55621.5,56163.9,56885.0,⋯
4,2018-01-01T01:00:00,2017-12-30T18:00:00,43779.1,44816.7,45475.0,45970.2,46373.0,46715.9,47016.5,47285.7,47530.5,47755.9,47965.3,48161.5,48346.4,48521.8,48688.8,48848.6,49002.0,49149.7,49292.3,49430

In [3]:
pre_unique_forecast_times = unique(load_forecast_raw.forecast_time);
pre_unique_issue_times = unique(load_forecast_raw.issue_time);

In [4]:
# CSV.write("02_pre_unique_forecast_times.csv", DataFrame(unique_forecast_times = pre_unique_forecast_times))
# CSV.write("02_pre_unique_issue_times.csv", DataFrame(unique_issue_times = pre_unique_issue_times))

In [5]:
CSV.write("02_raw_load_forecast.csv", load_forecast_raw)

"02_raw_load_forecast.csv"

the predata suggests that convert_hours_2018 is the issue...

now check that:

Now that we have changed naming conventions, lets do a full check across the different stages of the process workflow with one single dataframe 

In [6]:
# Load data
load_actuals = convert_hours_2018(load_actuals_raw);
load_actual_avg = convert_hours_2018(load_actual_avg_raw);
load_forecast = convert_hours_2018(load_forecast_raw, false);

# Solar data
solar_actuals = convert_hours_2018(solar_actuals_raw);
solar_actual_avg = convert_hours_2018(solar_actual_avg_raw);
solar_forecast_dayahead = convert_hours_2018(solar_forecast_dayahead_raw, false);
solar_forecast_2dayahead = convert_hours_2018(solar_forecast_2dayahead_raw, false);

# Wind data
wind_actuals = convert_hours_2018(wind_actuals_raw);
wind_actual_avg = convert_hours_2018(wind_actual_avg_raw);
wind_forecast_dayahead = convert_hours_2018(wind_forecast_dayahead_raw, false);
wind_forecast_2dayahead = convert_hours_2018(wind_forecast_2dayahead_raw, false);

In [7]:
wind_forecast_dayahead

Row,forecast_time,issue_time,p_1,p_2,p_3,p_4,p_5,p_6,p_7,p_8,p_9,p_10,p_11,p_12,p_13,p_14,p_15,p_16,p_17,p_18,p_19,p_20,p_21,p_22,p_23,p_24,p_25,p_26,p_27,p_28,p_29,p_30,p_31,p_32,p_33,p_34,p_35,p_36,p_37,p_38,p_39,p_40,p_41,p_42,p_43,p_44,p_45,p_46,p_47,p_48,p_49,p_50,p_51,p_52,p_53,p_54,p_55,p_56,p_57,p_58,p_59,p_60,p_61,p_62,p_63,p_64,p_65,p_66,p_67,p_68,p_69,p_70,p_71,p_72,p_73,p_74,p_75,p_76,p_77,p_78,p_79,p_80,p_81,p_82,p_83,p_84,p_85,p_86,p_87,p_88,p_89,p_90,p_91,p_92,p_93,p_94,p_95,p_96,p_97,p_98,⋯
,DateTime,DateTime,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,⋯
1,2017-12-31T18:00:00,2017-12-30T12:00:00,5375.28,5390.85,5405.27,5416.14,5598.63,5612.32,5629.37,5630.34,5831.55,5839.69,5840.0,5930.87,5998.46,6004.82,6022.48,6110.61,6156.18,6180.13,6253.17,6573.69,6676.18,6746.3,6752.9,6783.97,6892.0,7016.69,7083.21,7102.16,7165.32,7186.88,7195.03,7257.67,7313.68,7334.19,7360.11,7361.5,7398.83,7536.04,7546.97,7582.04,7608.55,7618.06,7647.63,7649.49,7649.74,7703.4,7707.09,7761.18,7900.63,7914.92,7921.56,8054.73,8066.51,8074.71,8081.28,8082.9,8126.53,8193.02,8199.29,8232.63,8260.07,8278.01,8298.24,8320.21,8324.29,8386.76,8407.7,8462.63,8478.51,8641.97,8701.69,8751.46,8751.97,8773.48,8822.54,8900.95,8954.41,8996.96,8997.98,9005.24,9052.34,9102.52,9131.08,9266.11,9308.38,9354.99,9383.08,9392.9,9641.58,9657.58,9671.19,9702.16,9811.37,9873.27,9903.07,10080.6,10135.2,10158.5,⋯
2,2017-12-31T19:00:00,2017-12-30T12:00:00,5318.03,5604.53,5675.43,5755.0,5877.84,5882.52,6242.31,6249.09,6286.43,6330.5,6380.25,6405.2,6473.85,6502.01,6516.66,6569.38,6572.36,6601.82,6649.18,6698.29,6728.3,6732.69,6752.0,6760.64,6785.99,6797.67,6807.54,6868.43,6883.11,6884.61,6907.94,6939.62,6949.0,7018.47,7087.45,7098.57,7099.07,7107.49,7127.93,7163.49,7163.88,7193.23,7221.23,7233.04,7329.47,7432.17,7442.55,7456.74,7461.67,7519.28,7522.3,7542.28,7597.75,7632.21,7632.27,7737.42,7765.16,7768.25,7838.13,7863.13,7887.08,7920.31,7944.79,7973.66,7982.01,7997.94,8006.85,8008.96,8014.71,8064.58,8088.6,8113.62,8137.09,8149.37,8181.15,8187.68,8217.72,8229.65,8306.46,8315.45,8378.56,8387.3,8407.1,8411.71,8413.48,8414.51,8434.6,8435.2,8439.51,8463.17,8528.05,8531.97,8568.97,8602.61,8609.0,8645.87,8646.73,8689.78,⋯
3,2017-12-31T20:00:00,2017-12-30T12:00:00,6025.79,6124.17,6127.07,6236.1,6248.33,6276.17,6310.73,6312.04,6314.03,6453.84,6484.45,6516.5,6560.56,6567.03,6573.47,6586.26,6599.72,6611.33,6634.79,6705.18,6722.37,6758.69,6769.75,6775.46,6911.91,6973.32,7003.56,7033.06,7100.43,7120.69,7138.85,7148.77,7161.24,7225.16,7369.23,7383.85,7450.04,7450.41,7467.28,7474.05,7486.97,7540.05,7547.97,7586.33,7611.76,7632.87,7673.7,7682.73,7695.08,7733.16,7817.26,7842.55,7856.55,7896.22,7918.53,7937.48,7937.74,7984.86,8017.31,8080.69,8106.76,8115.47,8126.83,8129.51,8143.23,8166.5,8190.72,8197.83,8216.56,8255.03,8258.84,8290.94,8368.11,8383.33,8395.79,8418.97,8419.02,8419.75,8420.58,8441.74,8474.17,8491.13,8500.32,8502.53,8505.73,8524.32,8534.19,8579.22,8623.07,8699.54,8727.69,8730.96,8738.03,8787.94,8792.16,8797.48,8921.79,9008.15,⋯
4,2017-12-31T21:00:00,2017-12-30T12:00:00,5136.96,5293.57,5298.07,5323.61,5435.21,5533.89,5536.2,5572.33,5621.77,5630.03,5714.93,5799.05,5834.51,5840.99,5946.49,6084.73,6116.99,6359.25,6377.73,6453.96,6719.36,6750.92,6768.61,

In [8]:
#=======================================================================
BIND HOURLY HISTORICAL DATA WITH FORECAST DATA
========================================================================#
#= The binding is made by ("forecast_time" = "time_index"). This causes the 
average actual value to be duplicated, which is desired, given the # of rows
in the load_forecast is double that of load_actual. To distinguish a 
one-day-ahead forecast from a two-day-ahead forecast, the column "ahead_factor"
is introduced. Bind the day-ahead and two-day-ahead forecasts for wind and solar
to get all the forecast data into one object as it is for load forecast =#
load_data = bind_historical_forecast(true,
    load_actual_avg,
    load_forecast);

solar_data = bind_historical_forecast(false,
    solar_actual_avg,
    solar_forecast_dayahead,
    solar_forecast_2dayahead);

wind_data = bind_historical_forecast(false,
    wind_actual_avg,
    wind_forecast_dayahead,
    wind_forecast_2dayahead);

In [9]:
load_forecast_raw;

Somewhere along the way, load_forecast_raw is re-written

Now the new theory is that the problem is in the groupby 48 ! because those DST things will have extra hours...

In [10]:
#=======================================================================
Landing probability
=======================================================================#
#= This section holds the calculation of the probability that the actual
value was equaled or superior than the forecast percentiles for a given
day. This is made possible by the estimation of an approximate CDF
computed on the forecast percentiles. Once estimated, this function is
used to find the "landing probability"; the prob. that the actual value
is equal or greater than a % percentage of the forecast percentile.
=#
#include(here("src", "functions", "fct_compute_landing_probability.jl"))
landing_probability_load = compute_landing_probability(load_data);
landing_probability_solar = compute_landing_probability(solar_data);
landing_probability_wind = compute_landing_probability(wind_data);

In [11]:
raw_unique_issue_times = sort(unique(load_forecast_raw.issue_time));
post_convert_unique_issue_times = sort(unique(load_forecast.issue_time));
post_bind_unique_issue_times = sort(unique(load_data.issue_time));
post_landing_unique_issue_times = sort(unique(landing_probability_load.issue_time));


In [12]:
test_issues = DataFrame(raw = raw_unique_issue_times, 
                        post_convert = post_convert_unique_issue_times, 
                        post_bind = post_bind_unique_issue_times,
                        post_landing = post_landing_unique_issue_times)

Row,raw,post_convert,post_bind,post_landing
,DateTime,DateTime,DateTime,DateTime
1,2017-12-29T18:00:00,2017-12-29T12:00:00,2017-12-29T12:00:00,2017-12-29T12:00:00
2,2017-12-30T18:00:00,2017-12-30T12:00:00,2017-12-30T12:00:00,2017-12-30T12:00:00
3,2017-12-31T18:00:00,2017-12-31T12:00:00,2017-12-31T12:00:00,2017-12-31T12:00:00
4,2018-01-01T18:00:00,2018-01-01T12:00:00,2018-01-01T12:00:00,2018-01-01T12:00:00
5,2018-01-02T18:00:00,2018-01-02T12:00:00,2018-01-02T12:00:00,2018-01-02T12:00:00
6,2018-01-03T18:00:00,2018-01-03T12:00:00,2018-01-03T12:00:00,2018-01-03T12:00:00
7,2018-01-04T18:00:00,2018-01-04T12:00:00,2018-01-04T12:00:00,2018-01-04T12:00:00
8,2018-01-05T18:00:00,2018-01-05T12:00:00,2018-01-05T12:00:00,2018-01-05T12:00:00
9,2018-01-06T18:00:00,2018-01-06T12:00:00,2018-01-06T12:00:00,2018-01-06T12:00:00


In [13]:
CSV.write("04_issue_times_full_check.csv", test_issues)

"04_issue_times_full_check.csv"

In [14]:
landing_probability_load

Row,issue_time,forecast_time,landing_probability,ahead_factor
,DateTime,DateTime,Float64,String
1,2017-12-29T12:00:00,2017-12-31T18:00:00,0.989899,two
2,2017-12-30T12:00:00,2017-12-31T18:00:00,0.545455,one
3,2017-12-29T12:00:00,2017-12-31T19:00:00,0.868687,two
4,2017-12-30T12:00:00,2017-12-31T19:00:00,0.525253,one
5,2017-12-29T12:00:00,2017-12-31T20:00:00,0.636364,two
6,2017-12-30T12:00:00,2017-12-31T20:00:00,0.666667,one
7,2017-12-29T12:00:00,2017-12-31T21:00:00,0.464646,two
8,2017-12-30T12:00:00,2017-12-31T21:00:00,0.474747,one
9,2017-12-29T12:00:00,2017-12-31T22:00:00,0.505051,two


In [15]:
#=======================================================================
ADJUST LANDING PROBABILITY DATAFRAME
=======================================================================#
lp_load = transform_landing_probability(landing_probability_load);
lp_solar = transform_landing_probability(landing_probability_solar);
lp_wind = transform_landing_probability(landing_probability_wind);

Let's test this and print out the df combined groupby and find the DST days.

In [16]:
x = copy(landing_probability_load);
# Sort data by issue time
sort!(x, :issue_time);
# Group data by issue time and count occurences in every group
df = combine(groupby(x, [:issue_time]), DataFrames.nrow => :count);

In [17]:
CSV.write("05_check_groupby_48.csv", df)

"05_check_groupby_48.csv"

Yes, the issue is that the daylight savings issue days have, for both the day of and day after, have 2 more hours of forecast apparently...

So how do i go about figuring this out...
* look at the original raw data in great detail
    * how does the raw data deal with DST? is it included? 
    * Are there multiple hours at the same time for those days? 
    * is the forecast still for 48 hours or do they extend the forecast for the extra hour(s)?

* maybe print out the forecasts... the 50 forecasts from the DST days? all need to do is groupby 50 etc

Some possible solutions:
* we could manually delete an extra 2 hours in these DST days

In [18]:
CSV.write("06_check_raw_data.csv", load_forecast_raw)

"06_check_raw_data.csv"

It does not appear to me that the raw data contains extra hours for DST...

Oh! what if the assumption that 1day and 2day are always alternating?

In [19]:
x = copy(landing_probability_load);
# Sort data by issue time
sort!(x, :issue_time);
# Group data by issue time and count occurences in every group
df = combine(groupby(x, [:issue_time]), DataFrames.nrow => :count);

dst_filter = filter(:count => ==(50), df)
forecast_time_dst = filter(row -> row.issue_time in dst_filter.issue_time, x).forecast_time


200-element Vector{DateTime}:
 2018-03-23T01:00:00
 2018-03-23T02:00:00
 2018-03-23T03:00:00
 2018-03-23T04:00:00
 2018-03-23T05:00:00
 2018-03-23T06:00:00
 2018-03-23T07:00:00
 2018-03-23T08:00:00
 2018-03-23T09:00:00
 2018-03-23T10:00:00
 ⋮
 2018-11-04T15:00:00
 2018-11-04T16:00:00
 2018-11-04T17:00:00
 2018-11-04T18:00:00
 2018-11-04T19:00:00
 2018-11-04T20:00:00
 2018-11-04T21:00:00
 2018-11-04T22:00:00
 2018-11-04T23:00:00

In [20]:
dst_filter

Row,issue_time,count
,DateTime,Int64
1,2018-03-22T13:00:00,50
2,2018-03-23T13:00:00,50
3,2018-11-01T13:00:00,50
4,2018-11-02T13:00:00,50


In [21]:
CSV.write("07_forecast_time_dst.csv", DataFrame(forecast_time_dst = forecast_time_dst))


"07_forecast_time_dst.csv"

There is no pattern to the errant or missing hours...